## Preamble

In [210]:
import os
import paramiko
from getpass import getpass

# Set up files to download

def user_file_download():
    """Ask the user what file they want to download and check if that file exists or not"""
    fileToGet = input("What file do you want to get from the remote machine? ")
    filename = fileToGet.split("/")
    filename = filename[-1]
    if os.path.exists("./remoteFiles/" + filename):
        while True:
            delete = input("A file with the same name already exists on your system. Would you like to delete it? (y/n)")
            if delete.lower() == "y":
                os.remove("./remoteFiles/" + filename)
                break
            elif delete.lower() == "n":
                break
    return fileToGet

def user_file_upload():
    """Get the file that the user wants to upload to the remote server"""
    while True:
        fileToPost = input("What file would you want to upload? ")
        if os.path.exists(fileToPost):
            return fileToPost
        else:
            try_again = input("That file does not exist on your system. Would you like to retry? (y/n)")
            if try_again.lower() == "y": 
                break
            else:
                exit()

def download_file(file):
    """Download the file over an already established ssh connection"""
    # Start SFTP session
    sftp = ssh.open_sftp()

    # Files to download
    filename = file.split('/')

    # Local path to store files
    local_path = "remoteFiles/" + filename[-1]

    sftp.get(file, local_path)

    #ssh.close()

def upload_file(file):
    """Upload a file to an already established ssh connection"""
    # Start SFTP session
    sftp = ssh.open_sftp()

    # Files to Upload 
    filename = file.split('/')

    # Remote path to put files
    remote_path = filename[-1]

    sftp.put(file, remote_path)

    #ssh.close()

def run_command(command, output=False):
    """Runs the command on the remote server"""
    stdin, stdout, stderr = ssh.exec_command(command)
    if output:
        print(stdout.readlines())
        #for line in iter(stderr.readline, "ls"):
        #    print(line, end="")
        #for line in iter(stdout.readline, "ls"):
        #    print(line, end="")
    

## Establish SSH Connection

In [211]:
# ask the user if they want to upload or download
"""
uod = input("Would you like to upload a file or download a file? (u/d)")
if uod.lower() == "d":
    fileToGet = user_file_download()
elif uod.lower() == "u":
    fileToPost = user_file_upload()
"""

#password prompt
thePass = getpass(prompt="Please enter SSH password: ")

#Host information
host = "192.168.6.71"
port = "2222"
username = "samuel.barrows"
password = thePass
#attempt the connection
try: 
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)
except paramiko.AuthenticationException:
    print("Authentication Failed")

## Run Scans

In [212]:
# Upload Kraken to the remote server
#upload_file("remoteFiles/kraken")

# Run our remote commands
try:
    # Kraken Scan
    print("### BEGIN KRAKEN SCAN ###")
    command = "./kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin | tee output.txt"
    stdin,stdout,stderr = ssh.exec_command("sudo -S -p '' " + command)
    stdin.write(thePass + "\n")
    stdin.flush()
    for line in stdout.readlines():
        print(line)

    print("### BEGIN LSOF ###")

    # lsof for PIDs 
    command = "cat output.txt | grep pid | cut -d ' ' -f 9 | cut -d '=' -f 2"
    stdin,stdout,stderr = ssh.exec_command(command)
    for line in stdout.readlines():
        stdin,stdout,stderr = ssh.exec_command("sudo -S -p '' lsof -p %s" % line.strip())
        stdin.write(thePass + "\n")
        stdin.flush()
        for line2 in stdout.readlines():
            print(line2)
    
except AttributeError as err:
    print(err)
    print("No open SSH connection")


### BEGIN KRAKEN SCAN ###
INFO[0000] Looking for configuration file with name config.yaml 

INFO[0000] No configuration file found, generating a default one... 

INFO[0000] Loading Yara rules...                        

INFO[0000] Scanning running processes...                

WARN[0004] DETECTION! Malicious process detected as foundGoBinary  pid=3138 process=ls

WARN[0004] DETECTION! Malicious process detected as foundGoBinary  pid=25680 process=snapd

INFO[0015] Scanning autoruns...                         

WARN[0016] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd

WARN[0016] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/bin/snap type=systemd

INFO[0016] Scanning the filesystem (this can take several minutes)... 

ERRO[0016] Some malicious artifacts have been detected on this system: 

ERRO[0016] Found detection for foundGoBinary            

ERRO[0016] Found detection for foundGoBinary            

ERRO[0016] 

## Download Suspicious Files

In [214]:
# Download Outputs
download_file("output.txt")
download_file("/home/p.dalton/test-lk/.d.log")
download_file("/home/p.dalton/test-lk/d.pid")

## Task 5

In [217]:
def sudo_run(command):
    stdin,stdout,stderr = ssh.exec_command("sudo -S -p '' %s" % command)
    stdin.write(thePass + "\n")
    stdin.flush()

    for line in stdout.readlines():
        print(line)

# Run our remote commands
try:
    print("### Running processes with p.dalton ###")

    # lsof for PIDs 
    command = "lsof -p 3138 | tee process.txt"
    sudo_run(command)

    # Scan /etc/passwd for any files that match the pattern of the threat group
    print("### /etc/passwd scan ###")
    command = "cat /etc/passwd | grep '^[a-z]\.[a-z]*' | tee etc-passwd.txt"
    sudo_run(command)

    # search for p.dalton in the auth.log
    print("### Auth.log scan ###")
    command = "cat /var/log/auth.log | grep p.dalton |tee auth-log.txt"
    sudo_run(command)

    sudo_run("mv /home/p.dalton/test-lk/ . && ls test-lk")
    
    # Download our output files

    download_file('auth-log.txt')
    download_file('etc-passwd.txt')
    download_file('process.txt')

except AttributeError as err:
    print(err)
    print("No open SSH connection")


### Running processes with p.dalton ###
COMMAND  PID     USER   FD      TYPE DEVICE SIZE/OFF   NODE NAME

ls      3138 p.dalton  cwd       DIR    8,1     4096 258130 /home/p.dalton/test-lk

ls      3138 p.dalton  rtd       DIR    8,1     4096      2 /

ls      3138 p.dalton  txt       REG    8,1  6597697 258133 /usr/local/bin/ls

ls      3138 p.dalton  DEL       REG    8,1            3453 /usr/lib/x86_64-linux-gnu/libc-2.31.so

ls      3138 p.dalton  DEL       REG    8,1            3466 /usr/lib/x86_64-linux-gnu/libpthread-2.31.so

ls      3138 p.dalton  DEL       REG    8,1            3449 /usr/lib/x86_64-linux-gnu/ld-2.31.so

ls      3138 p.dalton    0r      CHR    1,3      0t0      6 /dev/null

ls      3138 p.dalton    1w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log

ls      3138 p.dalton    2w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log

ls      3138 p.dalton    3r      CHR    1,3      0t0      6 /dev/null

ls      3138 p.dalton    4u      REG    

## Questions:

### What is the process name?

The process name in this case was something that I couldn't find, however, the PID is 3138 and it is a go program that is being run as a daemon. 

### What user is running the process?

p.dalton

### Has the user logged in before via SSH?

I do not believe so. The auth.log only goes back to March 6th for me.

### What is in the unauthorized user's home directory?

There is a directory called *test-lk*.

## Reflections

1. What is your conclusion?
    This machine is definetly comprimised with ThugStyle based on the activity on the machine. I couldn't get everything that I wanted because the auth.log file did not contain any useful information really.

2. What did you mlike the most and least about the assignment
    I liked the idea of performing a threat hunt, but it felt like I was missing something. I definetly missed a lot of information in my hunt and I'm not really sure why. I feel unsatisfied. 

3. What additional questions do you have?
    I think it would be neat to see your full code for this assignment, just to see how exactly I should be structuring it and what I should be looking for. I felt lost most of the time, but that might be just my current mental state.